In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("Processed_heart.csv")

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,0,140,289,0,0,172,0,0.0,0,0
1,49,1,1,160,180,0,0,156,0,1.0,1,1
2,37,0,0,130,283,0,1,98,0,0.0,0,0
3,48,1,2,138,214,0,0,108,1,1.5,1,1
4,54,0,1,150,195,0,0,122,0,0.0,0,0


In [4]:
X=df[['Age','Sex','ChestPainType','Cholesterol','MaxHR','RestingBP','FastingBS','RestingECG','ExerciseAngina', 'Oldpeak', 'ST_Slope']]
Y=df['HeartDisease']

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_std=sc.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_std, Y, test_size=0.30, random_state=0)

# Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV

DT = DecisionTreeClassifier(random_state=0)

params={'max_depth':np.arange(2,10),'criterion':['entropy','gini']}
kf=KFold(n_splits=5,shuffle=True,random_state=0)
GS=GridSearchCV(DT,params,cv=kf,scoring='f1_weighted')
GS.fit(X_std,Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=0),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='f1_weighted')

In [8]:
GS.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)

In [9]:
dt_reg=DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
dt_reg.fit(X_train,y_train)

y_train_pred = dt_reg.predict(X_train)
y_test_pred = dt_reg.predict(X_test)

from sklearn import metrics

acc_train=metrics.accuracy_score(y_train,y_train_pred)
print("Train Accuracy : ",acc_train*100)
acc_test=metrics.accuracy_score(y_test,y_test_pred)
print("Test Accuracy : ",acc_test*100)

from sklearn.model_selection import KFold,cross_val_score,GridSearchCV

#validate model performance using K-fold Cross Validation using roc scoring
kf=KFold(n_splits=5,shuffle=True,random_state=0)
scores=cross_val_score(dt_reg,X_std,Y,cv=kf,scoring='f1_weighted')

print("Bias Error : ",np.mean(scores))

print("Varience : ",np.std(scores, ddof= 1))

Train Accuracy :  89.8753894080997
Test Accuracy :  81.15942028985508
Bias Error :  0.8440378571748874
Varience :  0.03962329984900019


# Random Forest with RandomSearchCV

In [10]:
n_estimators = range(3,200,5) # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
RF=RandomForestClassifier(random_state=0)
rf_random = RandomizedSearchCV(estimator = RF,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=0, n_jobs = -1)

In [12]:
rf_random.fit(X_std, Y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': range(3, 200, 5)},
                   random_state=0, verbose=2)

In [13]:
print('Random grid: ', random_grid, '\n')
# print the best parameters
print('Best Parameters: ', rf_random.best_params_, ' \n')
print('Best Score: ', rf_random.best_score_, ' \n')
print('Best Estimatore : ', rf_random.best_estimator_ ,'\n')

Random grid:  {'n_estimators': range(3, 200, 5), 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]} 

Best Parameters:  {'n_estimators': 113, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 120, 'bootstrap': True}  

Best Score:  0.83976003801378  

Best Estimatore :  RandomForestClassifier(max_depth=120, max_features='sqrt', min_samples_leaf=4,
                       min_samples_split=10, n_estimators=113, random_state=0) 



In [14]:
rf_reg = RandomForestClassifier(max_depth=120, max_features='sqrt', min_samples_leaf=4,
                       min_samples_split=10, n_estimators=113, random_state=0) 

rf_reg.fit(X_train,y_train)

y_train_pred = rf_reg.predict(X_train)
y_test_pred = rf_reg.predict(X_test)


from sklearn import metrics

acc_train=metrics.accuracy_score(y_train,y_train_pred)
print("Train Accuracy : ",acc_train*100)
acc_test=metrics.accuracy_score(y_test,y_test_pred)
print("Test Accuracy : ",acc_test*100)

from sklearn.model_selection import KFold,cross_val_score,GridSearchCV

#validate model performance using K-fold Cross Validation using roc scoring
kf=KFold(n_splits=5,shuffle=True,random_state=0)
scores=cross_val_score(rf_reg,X_std,Y,cv=kf,scoring='f1_weighted')

print("Bias Error : ",np.mean(scores))

print("Varience : ",np.std(scores, ddof= 1))

Train Accuracy :  91.74454828660437
Test Accuracy :  85.14492753623189
Bias Error :  0.8731077556083943
Varience :  0.0186854168611879


# AdaBoosting with GridSearch Tunning

In [33]:
model = AdaBoostClassifier()

model.get_params().keys()

dict_keys(['algorithm', 'base_estimator', 'learning_rate', 'n_estimators', 'random_state'])

In [34]:
from sklearn.model_selection import RepeatedStratifiedKFold
model = AdaBoostClassifier()
# define the grid of values to search
ada_grid = dict()
ada_grid['n_estimators'] = range(170,180)
ada_grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the grid search procedure
ada_grid_search = RandomizedSearchCV(estimator = model,param_distributions = ada_grid,
               n_iter = 100, cv = cv, verbose=2, random_state=0, n_jobs = -1)
# execute the grid search
ada_grid_search.fit(X_std, Y)

Fitting 30 folds for each of 50 candidates, totalling 1500 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 50 is smaller than n_iter=100. Running 50 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1),
                   estimator=AdaBoostClassifier(), n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1.0],
                                        'n_estimators': range(170, 180)},
                   random_state=0, verbose=2)

In [35]:
print('Random grid: ', ada_grid, '\n')
# print the best parameters
print('Best Parameters: ', ada_grid_search.best_params_, ' \n')
print('Best Score: ', ada_grid_search.best_score_, ' \n')
print('Best Estimatore : ', ada_grid_search.best_estimator_ ,'\n')

Random grid:  {'n_estimators': range(170, 180), 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0]} 

Best Parameters:  {'n_estimators': 175, 'learning_rate': 0.1}  

Best Score:  0.8656872113393852  

Best Estimatore :  AdaBoostClassifier(learning_rate=0.1, n_estimators=175) 



In [36]:
ada_reg = ada_grid_search.best_estimator_

ada_reg.fit(X_train,y_train)

y_train_pred = ada_reg.predict(X_train)
y_test_pred = ada_reg.predict(X_test)


from sklearn import metrics

acc_train=metrics.accuracy_score(y_train,y_train_pred)
print("Train Accuracy : ",acc_train*100)
acc_test=metrics.accuracy_score(y_test,y_test_pred)
print("Test Accuracy : ",acc_test*100)

from sklearn.model_selection import KFold,cross_val_score,GridSearchCV

#validate model performance using K-fold Cross Validation using roc scoring
kf=KFold(n_splits=5,shuffle=True,random_state=0)
scores=cross_val_score(ada_reg,X_std,Y,cv=kf,scoring='f1_weighted')

print("Bias Error : ",np.mean(scores))

print("Varience : ",np.std(scores, ddof= 1))

Train Accuracy :  88.47352024922118
Test Accuracy :  84.78260869565217
Bias Error :  0.8679071250737351
Varience :  0.025073881241244007
